In [1]:
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/推荐系统/推荐系统算法复现/ml-100k")
import numpy as np
import pandas as pd

In [125]:
# 加载用户数据
users = pd.read_csv('./u.user', sep='|', names=['user_id', 'age', 'gender', 'occupation', 'zip_code'])

# 加载评分数据
ratings = pd.read_csv('./u.data', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])

# 加载电影信息
movies = pd.read_csv('./u.item', sep='|', encoding='latin-1', names=['item_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'])


In [126]:
# 去掉zip_code一列
users.drop('zip_code',axis = 1,inplace=True)

In [127]:
# 定义年龄转换函数，0-10岁为0，10-20岁为1，以此类推，最大取值为7
users.age = users.age//10
users

,user_id,age,gender,occupation
0,1,2,M,technician
1,2,5,F,other
2,3,2,M,writer
3,4,2,M,technician
4,5,3,F,other
...,...,...,...,...
938,939,2,F,student
939,940,3,M,administrator
940,941,2,M,student
941,942,4,F,librarian


In [128]:
one_hot_users = pd.get_dummies(users, columns=['age','gender','occupation'])

In [129]:
one_hot_users.shape

(943, 32)

In [130]:
one_hot_item = pd.concat([movies.iloc[:,0], movies.iloc[:,5:]],axis = 1)
one_hot_item

,item_id,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1678,1679,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1679,1680,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1680,1681,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [131]:
# 导入user-item交互矩阵
data = pd.read_csv("./user_item2.csv")
data.drop(data.columns[0], axis=1, inplace=True)
data

,user_id,item_id,rating
0,1,55,1
1,1,203,1
2,1,183,1
3,1,150,1
4,1,68,1
...,...,...,...
99995,943,427,1
99996,943,12,1
99997,943,284,0
99998,943,62,0


In [132]:
'''
此处共有三个dataframe，分别为one_hot_users，one_hot_item，data。
然后one_hot_users的主键是user_id，one_hot_item的主键是item_id，data的主键是user_id和item_id。
'''
merged_df = pd.merge(data, one_hot_users, on='user_id')
final_df = pd.merge(merged_df, one_hot_item, on='item_id')

In [133]:
final_df

,user_id,item_id,rating,age_0,age_1,age_2,age_3,age_4,age_5,age_6,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,55,1,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1,8,55,1,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,0
2,16,55,1,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
3,23,55,1,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,0
4,24,55,1,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,863,1679,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
99996,863,1678,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99997,863,1680,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
99998,896,1681,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [134]:
final_df.to_csv("one-hot_user&item_features.csv")